In [1]:
#!pip install haralyzer
#!pip install beautifulsoup4
#!pip install flashtext
!pip install nltk


You should consider upgrading via the '/data/jupyter-lab/provas-env/bin/python -m pip install --upgrade pip' command.


In [1]:
-1 * [1,1]

[]

In [2]:
from haralyzer import HarParser, HarPage
import glob
from http import HTTPStatus
from bs4 import BeautifulSoup
from flashtext.keyword import KeywordProcessor

from kafka import KafkaConsumer
import pymssql
import pandas as pd

import os
import json
import requests
import time

from elasticsearch import Elasticsearch
from elasticsearch.helpers import scan
from elasticsearch.helpers import bulk
from elasticsearch.helpers import BulkIndexError

from cryptography.fernet import Fernet # generar Fernet key
import configparser # Parsear el fichero de configuración

from datetime import datetime, timedelta, date

import warnings
warnings.filterwarnings('ignore')

In [3]:
def connect_to_elastic():
    elastic_url = 'https://10.210.4.119:9200'
    elastic_user = 'elastic'
    elastic_password = 'eilv2Dh04gbCBYAouc1o'

    es = Elasticsearch(
        elastic_url,
        #ca_certs= elastic_crt,
        basic_auth=(elastic_user, elastic_password),
        timeout=300,
        verify_certs=False
    )
    return es

def run_elastic_query(elastic_connection, query, index_name, page_size=10000, scroll="6m"):
    # Disable Elasticsearch scroll logs
    #logging.getLogger("elasticsearch").setLevel(logging.WARNING)

    
    page_size = page_size
    scroll_id = None

    response = scan(
            elastic_connection,
            query=query,
            index=index_name,
            scroll=scroll,
            size=page_size
    )

    results = []
    for hit in response:
        results.append(hit["_source"])

    return results


def insert_df_to_elastic(elastic_connection, df, index_name):
    # Disable Elasticsearch scroll logs
    #logging.getLogger("elasticsearch").setLevel(logging.WARNING)
    
    data_json = df.to_dict(orient='records')
    actions = [
        {
            "_index": index_name,
            "_source":  {key: value for key, value in record.items() if key != "id"},
            "_id": record["id"]
        }
        for record in data_json
    ]

    try:
        bulk(elastic_connection, actions, refresh=True)
        
    except BulkIndexError as e:
        for error in e.errors:
            print("Indexing failed for document:", error['index'])
            print("Reason:", error['error']['reason'])


def get_unique_idTest_from_elastic(index_name):
    aggs = {
    "unique_idTest": {
      "terms": {
        "field": "idTest.keyword",
          "size": 10000
          }
        }
     }
    result = es.search(index=index_name, size = 0, aggs=aggs)
    return result


def get_raw_asm_test_data(es, index_name, idTest, data_ini, data_fi):

    query = {
        "query": {
            "bool": {
              "must": [
                {
                  "match": {
                    "idTest": str(idTest)
                  }
                },
                {
                  "range": {
                    "fechaProcesado": {
                      "gte": data_ini,
                      "lte": data_fi 
                    }
                  }
                }
              ]
            }
        }
    }

    data = run_elastic_query(es, query, index_name)

    if(len(data) > 0):
        data_df = pd.DataFrame(data)

        if not 'idError' in data_df.columns:
            print("Sin idError")
            data_df['idError'] = 0
            print(data_df['idError'])
        if not 'pathError' in data_df.columns:
            print("Sin pathError")
            data_df['pathError'] = ""
        if not 'errorMsg' in data_df.columns:
            print("Sin errorMsg")
            data_df['errorMsg'] = ""
        if not 'urlFailed' in data_df.columns:
            print("Sin urlFailed")
            data_df['urlFailed'] = ""
        if not 'errorSel' in data_df.columns:
            print("Sin errorSel")
            data_df['errorSel'] = ""
        if not 'elementError' in data_df.columns:
            print("Sin elementError")
            data_df['elementError'] = ""
            
        data_df = data_df.rename(columns={'type': 'tipo'})
        data_df['fecha'] = data_df['fecha'].apply(lambda x: datetime.strptime(x, '%Y%m%d%H%M%S'))
        data_df['tiempoPaso'] = data_df['tiempoPaso'].astype(float)
        data_df['idError'] =  data_df['idError'].fillna(0)
        data_df['pathError'] = data_df['pathError'].fillna("")
        data_df['errorMsg'] = data_df['errorMsg'].fillna("")
        data_df['urlFailed'] = data_df['urlFailed'].fillna("")
        data_df['harFile'] = data_df['urlFailed'].fillna("")
        data_df['errorSel'] = data_df['errorSel'].fillna("")
        data_df['elementError'] = data_df['elementError'].fillna("")
        data_df['idError'] = data_df['idError'].astype(int)

    else:
        data_df = pd.DataFrame()
    return data_df
    

# Procesamiento de URLs

In [4]:
index_name = "raw-asm"
es = connect_to_elastic()
unique_idTest_list = get_unique_idTest_from_elastic(index_name)

In [5]:
# Definición de variables
datetime_ini = datetime.now() - timedelta(days=2)
datetime_ini = str(datetime_ini) 
datetime_ini = datetime.strptime(datetime_ini, "%Y-%m-%d %H:%M:%S.%f")
datetime_ini_str = datetime_ini.strftime("%Y-%m-%d %H:%M:%S")
print("datetime_ini", datetime_ini_str)

datetime_fi = datetime.now() - timedelta(days=1)
datetime_fi = str(datetime_fi)
datetime_fi = datetime.strptime(datetime_fi,"%Y-%m-%d %H:%M:%S.%f")
datetime_fi_str = datetime_fi.strftime("%Y-%m-%d %H:%M:%S")
print("datetime_fi", datetime_fi_str)

datetime_ini 2023-12-04 10:38:15
datetime_fi 2023-12-05 10:38:15


In [6]:
for hit in unique_idTest_list["aggregations"]["unique_idTest"]['buckets'][1:2]:
    idTest = hit['key']
    data_current_df = get_raw_asm_test_data(es, index_name, idTest, datetime_ini_str, datetime_fi_str)
    
    for index, row in data_current_df.iterrows():
        if row['urlFailed'] != "":
            #print(row['urlFailed'], row['errorSel'],row['errorMsg'],row['idPaso']  )
            row['errorSel'].split('?')
            url_error_dict = {
                "base_url": row['urlFailed'].split('?')[0],
                "url": row['urlFailed'],
                "error_selenium": row['errorSel'].replace("]", ""),
                "error_message": row['errorMsg'],
                "paso": row['idPaso']
            }
            print(url_error_dict)
            

Sin pathError
Sin errorMsg
Sin urlFailed
Sin errorSel
Sin elementError


1) Generar los datos
2) Generar el modleo
3) Clasificar 
4) Comparar

In [7]:
def get_match_percentage(value_0, value_x):
    try:
        total_percentage = float(value_x)/float(value_0)
        total_percentage = total_percentage * 100
    except:
        return 0
    else:
        return total_percentage


def get_text_category(text_from_html):
    
    text=str(text_from_html).lower()
    number_class_0 = len(all_keywords_processor.extract_keywords(text))
    number_class_1 = len(home_page_keywords_processor.extract_keywords(text))
    number_class_2 = len(login_keywords_processor.extract_keywords(text))
    number_class_3 = len(register_keywords_processor.extract_keywords(text))
    number_class_4 = len(services_keywords_processor.extract_keywords(text))
    number_class_5 = len(computer_keyword_processor.extract_keywords(text))
    number_class_6 = len(profile_keyword_processor.extract_keywords(text))
    number_class_7 = len(form_keyword_processor.extract_keywords(text))
    number_class_8 = len(banners_keyword_processor.extract_keywords(text))
    number_class_9 = len(contact_keyword_processor.extract_keywords(text))
    number_class_10 = len(security_keyword_processor.extract_keywords(text))
    number_class_11 = len(social_networks_keyword_processor.extract_keywords(text))
    number_class_12 = len(mobile_app_keyword_processor.extract_keywords(text))
    
    total_matches = number_class_0
    
    percentage_1 = float(get_match_percentage(number_class_0, number_class_1))
    percentage_2 = float(get_match_percentage(number_class_0, number_class_2))
    percentage_3 = float(get_match_percentage(number_class_0, number_class_3))
    percentage_4 = float(get_match_percentage(number_class_0, number_class_4))
    percentage_5 = float(get_match_percentage(number_class_0, number_class_5))
    percentage_6 = float(get_match_percentage(number_class_0, number_class_6))
    percentage_7 = float(get_match_percentage(number_class_0, number_class_7))
    percentage_8 = float(get_match_percentage(number_class_0, number_class_8))
    percentage_9 = float(get_match_percentage(number_class_0, number_class_9))
    percentage_10 = float(get_match_percentage(number_class_0, number_class_10))
    percentage_11 = float(get_match_percentage(number_class_0, number_class_11))
    percentage_12 = float(get_match_percentage(number_class_0, number_class_12))
    
    if number_class_0==0:
        category='None'
    else:
        percentages = [percentage_1, percentage_2, percentage_3, percentage_4, percentage_5,
               percentage_6, percentage_7, percentage_8, percentage_9, percentage_10,
               percentage_11, percentage_12]
        max_percentage = max(percentages)
        category = cartegories[percentages.index(max_percentage)]
    return category


In [8]:
# Página de Inicio
home_page_keywords = ["banca", "home", "promociones", "productos", "financieros", "bancarios", "servicios", "noticias", "banco", "ahorro", "préstamos"]

# Formularios de Inicio de Sesión
login_keywords = ["sesión", "login", "usuario", "contraseña", "password", "acceso", "credenciales", "identificador", "entrar"]

# Formularios de cierre de Sesión
logout_keywords = ["sesión", "logout", "cierre", "salir"]

# Sección de Registro
register_keywords = ["registro", "crear", "personal", "datos", "abrir"]

# Información de Productos y Servicios
services_keywords = ["ahorro", "cuenta", "corrientes", "préstamos", "tarjetas", "casa", "nómina", "fondos", "bolsa", "mercados", "sostenibilidad", "depósitos",
                     "crédito", "inversiones", "hipotecas", "donaciones", "jubilación", "inversión", "renting", "seguros", "salud", "vida", "hogar", "coche", "moto"]

# Calculadoras Financieras
computer_keywords = ["calculadora", "tasa", "interés", "pagos", "finanzas"]

# Zona de Usuarios Registrados
profile_keywords = ["acceso", "historial", "estado", "perfil", "configuración"]

# Formularios en Línea
form_keywords = ["transferencia", "error", "bad", "failed", "fondo", "pago", "préstamos", "form", "formulario", "actualización",
'input', 'rellenar', 'enviar', 'send']

# Banners y Anuncios
banners_keywords = ["ofertas", "promociones", "anuncios", "destacados", "noticias", "alertas"]

# Información de Contacto
contact_keywords = ["teléfono", "correo", "electrónico", "atención", "sucursales", "comunicación", "contacto",]

# Seguridad
security_keywords = ["seguridad", "privacidad", "protección"]


# Enlaces a Redes Sociales
social_networks_keywords = ["facebook", "twitter", "linkedIn", "redes", "sociales", "youtube", "instagram"]

# Broker
broker_app_keywords = ["bróker", "broker"]

# Móviles
mobile_app_keywords = ["móvil", "android", "mobile", "imagin", "apple", "app"]

keywords= home_page_keywords + logout_keywords + login_keywords + register_keywords + services_keywords + computer_keywords + profile_keywords + form_keywords +\
         banners_keywords + contact_keywords  + security_keywords + social_networks_keywords + mobile_app_keywords 
cartegories = ["Inicio", "Login", "Logout", "Registro", "Servicios", "Calculadora", "Perfil", "Formulario", "Banner", "Contacto", "Seguridad", "Redes sociales", "Móvil"]


In [ ]:
# Página de Inicio
home_page_keywords = ["banca", "home", "promociones", "productos", "financieros", "bancarios", "servicios", "noticias", "banco", "ahorro", "préstamos"]

# Formularios de Inicio de Sesión
login_keywords = ["sesión", "login", "usuario", "contraseña", "password", "acceso", "credenciales", "identificador", "entrar"]

# Formularios de cierre de Sesión
logout_keywords = ["sesión", "logout", "cierre", "salir"]

# Sección de Registro
register_keywords = ["registro", "crear", "personal", "datos", "abrir"]

# Información de Productos y Servicios
services_keywords = ["ahorro", "cuenta", "corrientes", "préstamos", "tarjetas", "casa", "nómina", "fondos", "bolsa", "mercados", "sostenibilidad", "depósitos",
                     "crédito", "inversiones", "hipotecas", "donaciones", "jubilación", "inversión", "renting", "seguros", "salud", "vida", "hogar", "coche", "moto"]


all_keywords_processor=KeywordProcessor()
for word in keywords:
    all_keywords_processor.add_keyword(word)
    
home_page_keywords_processor = KeywordProcessor()
for word in home_page_keywords:
    home_page_keywords_processor.add_keyword(word)
    
login_keywords_processor =KeywordProcessor()
for word in login_keywords:
    login_keywords_processor.add_keyword(word)

logout_keywords_processor =KeywordProcessor()
for word in logout_keywords:
    logout_keywords_processor.add_keyword(word)
    
register_keywords_processor=KeywordProcessor()
for word in register_keywords:
    register_keywords_processor.add_keyword(word)

services_keywords_processor=KeywordProcessor()
for word in services_keywords:
    services_keywords_processor.add_keyword(word)


In [9]:
all_keywords_processor=KeywordProcessor()
for word in keywords:
    all_keywords_processor.add_keyword(word)
    
home_page_keywords_processor = KeywordProcessor()
for word in home_page_keywords:
    home_page_keywords_processor.add_keyword(word)
    
login_keywords_processor =KeywordProcessor()
for word in login_keywords:
    login_keywords_processor.add_keyword(word)

logout_keywords_processor =KeywordProcessor()
for word in logout_keywords:
    logout_keywords_processor.add_keyword(word)
    
register_keywords_processor=KeywordProcessor()
for word in register_keywords:
    register_keywords_processor.add_keyword(word)

services_keywords_processor=KeywordProcessor()
for word in services_keywords:
    services_keywords_processor.add_keyword(word)

computer_keyword_processor = KeywordProcessor()
for word in computer_keywords:
    computer_keyword_processor.add_keyword(word)
    
profile_keyword_processor = KeywordProcessor()
for word in profile_keywords:
    profile_keyword_processor.add_keyword(word)
    
form_keyword_processor = KeywordProcessor()
for word in form_keywords:
    form_keyword_processor.add_keyword(word)
    
banners_keyword_processor = KeywordProcessor()
for word in banners_keywords:
    banners_keyword_processor.add_keyword(word)
    
contact_keyword_processor = KeywordProcessor()
for word in contact_keywords:
    contact_keyword_processor.add_keyword(word)
    
security_keyword_processor = KeywordProcessor()
for word in security_keywords:
    security_keyword_processor.add_keyword(word)
    
social_networks_keyword_processor = KeywordProcessor()
for word in social_networks_keywords:
    social_networks_keyword_processor.add_keyword(word)
    
mobile_app_keyword_processor = KeywordProcessor()
for word in mobile_app_keywords:
    mobile_app_keyword_processor.add_keyword(word)


In [10]:
xxx

NameError: name 'xxx' is not defined

In [ ]:
url_classficaion_list = []
processed_url = {}
# Recorrer los test
for hit in unique_idTest_list["aggregations"]["unique_idTest"]['buckets']:
    idTest = hit['key']
    # Recorrer los test
    data_current_df = get_raw_asm_test_data(es, index_name, idTest, datetime_ini_str, datetime_fi_str)
    
    for index, row in data_current_df.iterrows():
        # Filtrar las filas que tengan valor para el campo urlFailed
        if row['urlFailed'] != "":
            main_url = row['urlFailed']
            base_url =  main_url.split('?')[0]
            # Si todavía no se ha procesado la url, se procesa
            if not base_url in processed_url.keys():
                t0 = time.time()
                try:
                    # Recogemos el html, gurdando el tiempo de respuesta para detenr el programa temporalmente en función del mismo
                    # De esta forma se evita saturar la web consultada
                    url_response = requests.get(main_url)
                    url_response = requests.get(main_url, verify=False, cookies=url_response.cookies)
                    response_delay = time.time() - t0
                except:
                    print("error")
                    break
                # Obtener el contenido
                soup = BeautifulSoup(url_response.content)
                main_div = soup.find_all(class_="main")
                if(len(main_div) > 0):
                    main_div = main_div[0]
                else:
                    main_div = soup
                texts = main_div.findAll(text=True)
                text_from_html = ' '.join(texts)
                # Clasificar según las categorías
                category = get_text_category(text_from_html)
                url_classficaion_dict = {
                    "tokenized_source": text_from_html[0:800],
                    "category": category,
                    "url": main_url
                }
                processed_url[base_url] = category
                url_classficaion_list.append(url_classficaion_dict)
    
                time.sleep(1 * response_delay)



print("------END-------")
today = str(datetime.today().strftime('%Y-%m-%d'))
url_classficaion_df = pd.DataFrame(url_classficaion_list)
url_classficaion_df.to_csv(f"url_classficaion_data_{today}_1.csv", index=False)

In [ ]:
url_list = [
    "https://www.caixabank.es/particular/prestamos-personales.html",
    "https://www.caixabank.es/particular/hipotecas.html",
    "https://www.caixabank.es/particular/tarjetas-bancarias.html",
    "https://www.caixabank.es/particular/ahorro.html",
    "https://www.caixabank.es/particular/prestamos-personales.html",
    "https://www.caixabank.es/particular/hipotecas.html",
    "https://www.caixabank.es/particular/tarjetas-bancarias.html",
    "https://www.caixabank.es/particular/ahorro.html",
    "https://www.caixabank.es/particular/cuentas-bancarias.html",
    "https://www.caixabank.es/particular/planes-de-pensiones.html",
    "https://www.caixabank.es/particular/fondos-de-inversion.html",
    "https://www.caixabank.es/particular/bolsa-mercados.html",
    "https://www.caixabank.es/particular/sostenibilidad.html",
    "https://www.caixabank.es/particular/seguros.html",
    "https://www.caixabank.es/particular/seguros/seguros-hogar.html",
    "https://www.caixabank.es/particular/seguros/seguros-vida.html",
    "https://www.caixabank.es/particular/seguros/seguros-coche-moto.html",
    "https://www.caixabank.es/particular/seguros/seguros-salud-adeslas.html",
    "https://www.caixabank.es/particular/seguros/seguros-ahorro.html",
    "https://www.caixabank.es/particular/movilidad.html"
    
]

In [21]:
#main_url= "https://www.rtve.es/play/"
main_url = "https://www.generali.es/landings/seguros-generali.html?client=false&section=section-no-client&&utm_source=google&utm_medium=cpc&utm_campaign=ES_ES_AO_ADW_._MARCA-PURA_._._MARCA-PURA_._._SEARCH_MARCA-PURA_MARCA-PURA&utm_term=generali%20seguros&gad_source=1&gclid=CjwKCAiA1MCrBhAoEiwAC2d64e6IkP7L40Jj4Nm1KPe8XS3yTBsi3hon3IRwWDv2nOJa_oKJ-LZ9JxoCLxUQAvD_BwE&gclsrc=aw.ds"
#main_url = "https://catsalut.gencat.cat/ca/inici/"

url_response = requests.get(main_url)
url_response = requests.get(main_url, verify=False, cookies=url_response.cookies)

soup = BeautifulSoup(url_response.content)

# Encuentra todos los elementos 'a' (enlaces) con el atributo 'href'
all_links = soup.find_all('a', href=True)

# Extrae las URLs de los enlaces
urls = [link['href'] for link in all_links]
#urls = ['https://www.caixabank.es' + url if url.startswith('/') else url for url in urls]

print(len(urls))
urls = [url for url in urls if 'https' in url]
print(len(urls))


34
24


In [13]:
urls[1:10]

['https://secure2.rtve.es/usuarios/contenidos/mi-tv/',
 'https://secure2.rtve.es/usuarios/contenidos/mi-radio/',
 'https://secure2.rtve.es/usuarios/cuenta/mis-datos/',
 'https://secure2.rtve.es/usuarios/cuenta/mis-perfiles/',
 'https://secure2.rtve.es/usuarios/cuenta/mi-suscripcion/',
 'https://secure2.rtve.es/usuarios/acceso/login-tv/',
 'https://www.rtve.es/play/videos/duos-increibles/',
 'https://www.rtve.es/play/videos/duos-increibles/programa-10-final/7024226/',
 'https://secure2.rtve.es/usuarios/acceso/login/']

In [14]:
import string 
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
stopwords_spanish = set(stopwords.words('spanish'))


def clean_html_data(text):
    # Eliminar saltos de línea
    text = text.replace('\r\n', ' ')
    text = text.replace('\n\n', ' ') 
    text = text.replace('\n', ' ')
    text = text.replace('\r', ' ') 

    # Eliminar espacios innecesarios
    text = re.sub('\s\s+', ' ', text)
    
    # Eliminar caracteres
    #text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    text = re.sub(r'[^\w\sáéíóúÁÉÍÓÚüÜñÑ]', '', text)

    # ELiminar HTML PUBLIC W3CDTD HTML 40 TransitionalEN
    text = text.replace('HTML PUBLIC W3CDTD HTML 40 TransitionalEN', '')
    # Pasar a minusculas
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))

    # Remove storpwords 
    # Tokenize the text
    words = word_tokenize(text)
    
    # Remove stopwords
    filtered_words = [word for word in words if word.lower() not in stopwords_spanish]
    
    # Join the filtered words to form the processed text
    processed_text = ' '.join(filtered_words)
    return processed_text


    

In [11]:
import pandas as pd
data_1 = pd.read_csv('url_classficaion_data_2023-12-05_3.csv', encoding='utf-8')
data_2 = pd.read_csv('url_classficaion_data_2023-12-05_1.csv', encoding='utf-8')
data_3 = pd.read_csv('url_classficaion_data_2023-12-06_1.csv', encoding='utf-8')
#data_4 = pd.read_csv('url_classficaion_data_2023-12-06_2.csv', encoding='utf-8')
#data_5 = pd.read_csv('url_classficaion_data_2023-12-06_3.csv', encoding='utf-8')
print(data_3['tokenized_source'][1:10])

# Concatenate along rows (axis=0)
result_df = pd.concat([data_1, data_2,data_3, data_4, data_5 ], axis=0, ignore_index=True)

# Save to CSV
result_df.to_csv('url_classification_data.csv', index=False)

1    html tv html cabecera nueva rtve lista inicio ...
2    html radio html cabecera nueva rtve lista inic...
3    html edición cuenta html cabecera nueva rtve c...
4                                            not found
5    html suscripción rtveplay html cabecera nueva ...
6    html asocia dispositivo cuenta usuario html ca...
7    html if lte ie 9 lista iniciar sesión tv radio...
8    html if lte ie 9 lista iniciar sesión tv radio...
9    crear cuenta accede contenido rtve continúa vi...
Name: tokenized_source, dtype: object


NameError: name 'data_4' is not defined

In [ ]:
data = pd.read_csv('url_classification_data_complete.csv', encoding='utf-8')
urls = data['url']
new_data = []
for index, row in data.iterrows():
    text = row['tokenized_source']
    text_from_html = BeautifulSoup(text, "lxml").text
    text_from_html = clean_html_data(text_from_html)
    dict = {"tokenized_source":text_from_html,
           "category": row['category'], 
           "url": row['url']
           }
    new_data.append(dict)
url_classficaion_df = pd.DataFrame(new_data)
url_classficaion_df.to_csv(f"url_classficaion_data_{today}_3.csv", index=False)  

In [22]:
url_classficaion_list = []
processed_url = {}

for main_url in urls:
    base_url =  main_url.split('?')[0]
    if not base_url in processed_url.keys():
        #print("New url")
        t0 = time.time()
        try:
            url_response = requests.get(main_url)
            url_response = requests.get(main_url, verify=False, cookies=url_response.cookies)
            response_delay = time.time() - t0
        except:
            print("error")
            break
        
        soup = BeautifulSoup(url_response.content)
        
        main_div = soup.find_all(class_="main")
        if(len(main_div) > 0):
            main_div = main_div[0]
        else:
            main_div = soup
        #texts = main_div.get_text(separator='')
        texts = main_div.findAll(text=True)
        text_from_html = ' '.join(texts)
        text_from_html = BeautifulSoup(text_from_html, "lxml").text
        text_from_html = clean_html_data(text_from_html)
        if(len(text_from_html) > 20000):
            print("KO")
            text_from_html = text_from_html[0:20000]
        category = get_text_category(text_from_html)
        url_classficaion_dict = {
            "tokenized_source": text_from_html,
            "category": category,
            "url": main_url
        }
        processed_url[base_url] = category
        url_classficaion_list.append(url_classficaion_dict)

        #print(f"sleep {response_delay}")
        time.sleep(1 * response_delay)



print("------END-------")
today = str(datetime.today().strftime('%Y-%m-%d'))
url_classficaion_df = pd.DataFrame(url_classficaion_list)
url_classficaion_df.to_csv(f"url_classficaion_data_{today}_3.csv", index=False)

KO
KO
------END-------


In [ ]:
len(unique_idTest_list["aggregations"]["unique_idTest"]['buckets'])

In [ ]:
#processed_url

In [ ]:
url_classficaion_df = pd.DataFrame(url_classficaion_list)
url_classficaion_df.to_csv("url_classficaion_data.csv", index=False)

In [ ]:
data_df = url_classficaion_df[pd.notnull(url_classficaion_df['tokenized_source'])]
data_df = data_df[data_df.category!='None']
len(data_df)

In [ ]:
train_data = []
for index,row in data_df.iterrows():
    train_data.append({"class":row["category"], "sentence":row["tokenized_source"]})

In [ ]:
words = []
classes = []
documents = []
ignore_words = ['?']
# loop through each sentence in our training data
for pattern in train_data:
    # tokenize each word in the sentence
    w = nltk.word_tokenize(pattern['sentence'])
    # add to our words list
    words.extend(w)
    # add to documents in our corpus
    documents.append((w, pattern['class']))
    # add to our classes list
    if pattern['class'] not in classes:
        classes.append(pattern['class'])

# stem and lower each word and remove duplicates
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = list(set(words))

# remove duplicates
classes = list(set(classes))

print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)

In [ ]:
#creating a variable
main_url = "https://www4.caixabank.es/apl/donativos/solicitarCargo_es.html?stateName=showImporteYOpcionFiscal&JSESSIONID=kc1mke96lm6V5XKhujYMguO&tkn_ctx=Npx11Be6RG9svIKgT6GzpTW0Tbt93teBeeXBT3mAeTSSyXu5SH"
url_response = requests.get(main_url)
url_response = requests.get(main_url, verify=False, cookies=url_response.cookies)
#create a beautifulsoup object
soup = BeautifulSoup(url_response.content)
#print soup
#print(mark_soup.prettify())



In [ ]:
main_div = soup_html.find_all(class_="main")[0]
texts = main_div.findAll(text=True)                 #find all text
text_from_html = ' '.join(texts)
catgeory = get_text_category(text_from_html)


In [ ]:
def find_class:
    
        
    return category

In [ ]:
main_div = soup_html.find_all(class_="main")[0]
p_list = []
title_list = []
href_list = []
link_list = []

for item in main_div.find_all(['div', 'a']):
    #print("-----------item", item)
    title_list.append(item.get("title"))
    href_list.append(item.select("href"))
    text_list.append(item.get_text())
    print(item.select("p"))
#print(link_list)

In [ ]:
main_div = soup_html.find_all(class_="main")[0]
links_soup = main_div.select("a")
title_list = []
links_list = []
text_list = []
for item in links_soup : 
    print(item)
    #title_list.append(item.get("title"))
    #links_list.append(item.get("href"))
    labels_list.append(item.get_text())
#links_list
labels_list

In [ ]:
title_list = []
links_list = []

for eachitem in soup_html.find_all(class_="main") :    
    title_list.append(eachitem.get("title"))
    links_list.append(eachitem.get("href"))
title_list
links_list

# Procesamiento de HARS

In [ ]:
def traducir_codigo_estado(codigo_estado):
    try:
        # Obtener el mensaje asociado al código de estado
        mensaje = HTTPStatus(codigo_estado).phrase
        return f'{codigo_estado} {mensaje}'
    except ValueError:
        # En caso de que el código de estado no sea válido
        return f'Código de estado no válido: {codigo_estado}'

In [ ]:
directorio = '/data/asm-code/asm_kafka_collector/hars'

# Especificar el patrón de búsqueda (puedes usar '*' para todos los archivos)
patron = '*.har'  # Por ejemplo, busca todos los archivos con extensión .txt

# Construir la ruta completa del patrón
patron_completo = os.path.join(directorio, patron)

# Obtener la lista de archivos que coinciden con el patrón
archivos = glob.glob(patron_completo)
print("patron_completo", patron_completo)
har_list = []
# Iterar sobre la lista de archivos
for archivo in archivos:
    #print("archivo", archivo)
    with open(archivo, 'r') as f:
        har_parser = HarParser(json.loads(f.read())['HARFile'])
    data = har_parser.har_data
    print(archivo)
    entry_data = har_parser.har_data["entries"][0]
    #print(entry_data.keys())
    #print(entry_data)
    #print(entry_data["response"]["status"], traducir_codigo_estado(entry_data["response"]["status"]))
    har_data_dict= {
        "server_response_code": entry_data["response"]["status"],
        "server_response_message": traducir_codigo_estado(entry_data["response"]["status"]),
        "status_text": entry_data["response"]["statusText"],
        "url": entry_data["request"]["url"],
        "date": entry_data["startedDateTime"].split('.')[0],
        "id": (archivo.split('/')[-1]).split('.')[0]
    }
    
    #fecha_hora_objeto = datetime.fromisoformat(har_data_dict["date"])
    har_list.append(har_data_dict)
    #print(har_data_dict)
    
    
har_list

In [ ]:

with open('sample.har', 'r') as f:
    har_parser = HarParser(json.loads(f.read()))